In [2]:
import streamlit as st
import pandas as pd
import altair as alt
from sqlalchemy import create_engine
from scripts.config import DB_CONNECTION_STRING

In [7]:
engine = create_engine(DB_CONNECTION_STRING)
city = "Tokyo"
sql = "SELECT * FROM weather_daily WHERE city = %s"
df_daily = pd.read_sql(sql=sql, con=engine, params=(city,))
df_daily

,date,city,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0
...,...,...,...,...,...
2214,2026-01-23 00:00:00+00:00,Tokyo,8.10,-2.95,0.0
2215,2026-01-24 00:00:00+00:00,Tokyo,10.35,0.45,0.2
2216,2026-01-25 00:00:00+00:00,Tokyo,7.20,-0.50,0.0
2217,2026-01-26 00:00:00+00:00,Tokyo,7.90,1.35,0.0
